In [6]:
import requests
import io
from datetime import timedelta
import uuid

import numpy as np
import pandas as pd 

import random

In [2]:
# TODO: не работает, разобраться 
# YEARS = [2019, 2020]

# def donwload_nyc_data(taxi_type):
#   trips = pd.DataFrame()

#   for year in YEARS:
#     for month in range(1, 13):
#       url = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year}-{f"0{month}" if month < 10 else month}.csv'
#       print(url)

#       s=requests.get(url).content
#       df = pd.read_csv(io.StringIO(s.decode('utf-8'))).sample(n=10000)

#       trips = pd.concat([trips, df], ignore_index=True)
#       break
#   return trips

In [20]:
trips_table = pd.read_csv('simulated_trips.csv')

In [21]:
trips_table[trips_table['assigned'] == True]

,Unnamed: 0,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,...,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,trip_id,assigned
0,126,HV0003,B03404,B03404,2024-01-01 00:30:00,2024-01-01 00:13:47,2024-01-01 00:28:26,2024-01-01 00:46:39,141,90,...,0.0,0.00,31.50,N,N,N,N,N,760b1722-b77b-4320-bacd-660e7620dbdc,True
1,233,HV0003,B03404,B03404,2024-01-01 01:00:00,2024-01-01 00:41:55,2024-01-01 00:47:33,2024-01-01 01:18:28,107,151,...,0.0,7.44,31.77,N,N,N,N,N,5331daf8-20e9-487a-bb56-75a9b1f9e940,True
2,246,HV0005,B03406,NaN,2024-01-01 00:45:00,NaN,2024-01-01 00:25:21,2024-01-01 00:41:00,36,17,...,0.0,7.79,20.26,N,N,N,N,N,0deee488-c026-4634-8fcc-1acc3d5e804d,True
3,319,HV0003,B03404,B03404,2024-01-01 01:00:00,2024-01-01 00:47:44,2024-01-01 00:59:08,2024-01-01 01:24:01,7,258,...,0.0,4.84,32.53,N,N,N,N,N,67842685-21d0-4208-9180-5761356f1c6e,True
4,587,HV0005,B03406,NaN,2024-01-01 00:45:00,NaN,2024-01-01 00:25:49,2024-01-01 00:40:52,114,246,...,0.0,0.00,22.14,N,N,N,N,N,3ed22b57-29ac-49ff-9afc-c9f8264d4aaa,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29019,4085381,HV0005,B03406,NaN,2024-01-08 00:00:00,NaN,2024-01-07 23:45:02,2024-01-07 23:57:01,233,186,...,0.0,0.00,8.78,N,N,N,N,N,019e60c2-dcc0-4bdf-9629-4bbac5c17c47,True
29020,4089474,HV0005,B03406,NaN,2024-01-07 23:25:00,NaN,2024-01-07 23:09:30,2024-01-07 23:34:32,97,197,...,0.0,0.00,31.97,N,N,N,N,N,3ce6fae2-64aa-4a1f-8039-4ed1b80472e6,True
29021,4091446,HV0005,B03406,NaN,2024-01-07 23:30:00,NaN,2024-01-07 23:12:34,2024-01-07 23:37:14,230,265,...,0.0,0.00,25.68,N,N,N,N,N,a7f64c9f-3af6-4f63-b64b-7283b56f11b8,True
29022,4092059,HV0005,B03406,NaN,2024-01-07 23:30:00,NaN,2024-01-07 23:15:00,2024-01-07 23:31:06,26,149,...,0.0,0.00,14.21,N,N,N,N,Y,c49e8677-df88-4586-a83c-58a883b4cbc3,True


# 0. Preprocess data - trips_table

In [5]:
trips_table = pd.read_parquet('fhvhv_tripdata_2024-01 (1).parquet')
trips_table.sample(100).to_csv('sample.csv')
# trips_table = pd.read_csv('sample.csv')

In [13]:
trips_table = trips_table[trips_table['shared_request_flag'] == 'N']
trips_table = trips_table[trips_table['wav_request_flag'] == 'N']
trips_table = trips_table[trips_table['access_a_ride_flag'] == 'N'] 

In [38]:
trips_table.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [20]:
(trips_table['pickup_datetime'] - trips_table['request_datetime']).describe()

count                  18872924
mean     0 days 00:04:03.833627
std      0 days 00:03:11.087633
min           -1 days +23:18:42
25%             0 days 00:02:33
50%             0 days 00:03:37
75%             0 days 00:05:07
max             0 days 20:30:37
dtype: object

In [22]:
trips_table = trips_table[trips_table['request_datetime'] > trips_table['pickup_datetime']]

In [31]:
trips_table.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [32]:
# filtering outliers
trips_table = trips_table[(trips_table['trip_time'] >= trips_table['trip_time'].quantile(0.05)) \
                    & (trips_table['trip_time'] <= trips_table['trip_time'].quantile(0.95)) \
                    & (trips_table['trip_miles'] >= trips_table['trip_miles'].quantile(0.05)) \
                    & (trips_table['trip_miles'] <= trips_table['trip_miles'].quantile(0.95)) \
                    ]

In [36]:
trips_table['base_passenger_fare'].describe()

count    153707.000000
mean         40.710398
std          22.060257
min           0.000000
25%          23.200000
50%          39.040000
75%          52.750000
max         255.750000
Name: base_passenger_fare, dtype: float64

In [127]:
trips_table['trip_id'] = [str(uuid.uuid4()) for _ in range(len(trips_table))]

/var/folders/xs/_0l9xrzj3tz9bsyngdnyxmhr0000gn/T/ipykernel_16744/3005511810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_table['trip_id'] = [str(uuid.uuid4()) for _ in range(len(trips_table))]


# 1. Simulation

## 1. 1. Intitial values

In [142]:
fleet_size = 50

charging_time = 60 * 60   # time to charge fo a full battery in seconds
distance_per_charge = 300  # range a car that travle on full battery in km
speed_of_charging = distance_per_charge

charging_stations_count = 10
charging_station_capacity = 5
electricity_consumption_per_km = 1  # in units of battery level consumed per km
max_wait_time = 900  # Maximum wait time for a trip in seconds (15 minutes)
co2_per_km = 0.2  # CO2 emissions in kg per km for traditional taxis


# mean distance and time between locations (used to calculate eta)
# can be also devided by traffic/ or high/low demand - next step for enhancement
mean_distance_time = trips_table.groupby(['PULocationID', 'DOLocationID']).agg({
    'trip_time': 'mean',
    'trip_miles': 'mean'
}).reset_index()

# initialize charging stations within each location_id
max_location_id = trips_table["PULocationID"].max()

charging_stations = pd.DataFrame({
    "location_id": random.choices(range(1, max_location_id + 1), k=charging_stations_count),
    "occupied_slot": [[0 for _ in range(charging_station_capacity)] for _ in range(charging_stations_count)],
    "waiting_to_be_charged": [[] for _ in range(charging_stations_count)],
    "estimated_time_to_charge": [[0 for _ in range(charging_station_capacity)] for _ in range(charging_stations_count)],
    "station_id": [str(uuid.uuid4()) for _ in range(charging_stations_count)]
    
})


# Initialize Fleet
fleet = pd.DataFrame({
    "location_id": random.choices(range(1, max_location_id + 1), k=fleet_size),
    "battery_level": [distance_per_charge] * fleet_size,  # Full battery
    "status": ["idle"] * fleet_size,  # idle, en_route_to_client, en_route_with_client, charging
    "current_trip_id": [None] * fleet_size,
    "eta_to_pickup": [0] * fleet_size,  # Estimated time to pickup
    "car_id": [str(uuid.uuid4()) for _ in range(len(fleet))]
})

## 1.2. Functions

In [146]:
def assign_rides(trips_data, current_time):
    """Assign cars to trips based on proximity, battery level, and wait time."""
    for idx, trip in trips_data.iterrows():
        if trip["assigned"] or (current_time - trip["request_datetime"]).total_seconds() > max_wait_time:
            continue

        required_battery = trip["trip_distance"] * electricity_consumption_per_km

        available_cars = fleet[(fleet["status"] == "idle") & (fleet["battery_level"] > required_battery)].copy()
        
        pick_up_location = trip['PULocationID']
        available_cars = available_cars.merge(mean_distance_time[mean_distance_time['DOLocationID'] == pick_up_location], 
                             left_on='location_id', 
                             right_on='PULocationID').rename(columns={'trip_time': 'eta_to_pickup',
                                                                     'trip_miles': 'miles_to_pickup'})
        
        # additional filtereing by consumption of battery needed to pickup client
        available_cars['required_battery_to_pickup_client'] = available_cars['miles_to_pickup'] * electricity_consumption_per_km
        available_cars['required_batter_to_make_trip'] = available_cars['required_battery_to_pickup_client'] + required_battery
        available_cars = available_cars[available_cars['battery_level'] > available_cars['required_batter_to_make_trip']]

        
        if available_cars.empty:
            continue

        closest_car = available_cars.nsmallest(1, "miles_to_pickup").iloc[0]

        # Assign car to the trip
        fleet.loc[closest_car.name, "status"] = "en_route_to_client"
        fleet.loc[closest_car.name, "current_trip_id"] = trip['trip_id']
        fleet.loc[closest_car.name, "eta_to_pickup"] = current_time + timedelta(seconds=closest_car['eta_to_pickup'])
        fleet.loc[closest_car.name, "battery_level"] -= closest_car['required_batter_to_make_trip']
        trips_data.loc[idx, "assigned"] = True

In [147]:
def update_car_status(trips_data, current_time):
    """Update car statuses and handle charging."""
    
    for idx, car in fleet.iterrows():
        if car["status"] == "en_route_to_client" and current_time >= car["eta_to_pickup"]:
            continue

        if car["status"] == "en_route_with_client":
            trip = trips_data[trips_data['trip_id'] == car["current_trip_id"]]
            if current_time >= trip["dropoff_datetime"]:
                fleet.loc[idx, "location_id"] = trip["DOLocationID"]
                fleet.loc[idx, "status"] = "idle"
                fleet.loc[idx, "current_trip_id"] = None

        if car["status"] == "charging":
            fleet.loc[idx, "battery_level"] += speed_of_charging  # Increment charging
            station_id = car["charging_station_id"]
            
            charging_station_idx = charging_stations[charging_stations['station_id'] == station_id].index[0]
            charging_station = charging_stations[charging_stations['station_id'] == station_id].iloc[0]
            
            charging_slot = charging_station['occupied_slot'].index(car['car_id'])
            charging_stations.loc[charging_station_idx, 'estimated_time_to_charge'][charging_slot] -= car['battery_level'] / speed_of_charging

            if car["battery_level"] >= distance_per_charge:
                fleet.loc[idx, "status"] = "idle"
                charging_stations.loc[charging_station_idx, 'occupied_slot'][charging_slot] = 0
                charging_stations.loc[charging_station_idx, 'estimated_time_to_charge'][charging_slot] = 0
                
                if charging_station["waiting_to_be_charged"]:
                    next_car = charging_station.loc[charging_station_idx, "waiting_to_be_charged"].pop(0)
                    charging_stations.loc[charging_station_idx, 'occupied_slot'][charging_slot] = next_car
                    charging_stations.loc[charging_station_idx, 'estimated_time_to_charge'][charging_slot] = charging_time


        if car["battery_level"] <= 20 and car["status"] == "idle":
            assign_to_charging(idx)

In [148]:
def assign_to_charging(car_id):
    """Assign car to the nearest available charging station."""
    available_stations = charging_stations.copy()

    car_location = fleet[fleet['car_id'] == car_id]
    
    available_stations = available_stations.merge(mean_distance_time[mean_distance_time['POLocationID'] == car_location], 
                             left_on='location_id', 
                             right_on='DOLocationID').rename(columns={'trip_time': 'trip_time_to_charger',
                                                                     'trip_miles': 'trip_miles_to_charger'})
    
    available_stations['required_battery_to_station'] = available_stations['trip_miles_to_charger'] * electricity_consumption_per_km
    available_stations = available_stations[available_stations['required_battery_to_station'] <\
                                                    fleet[fleet['car_id'] == car_id]['battery_level'].iloc[0]]
    
    available_stations["estimated_wait_time"] = available_stations['estimated_time_to_charge'].apply(lambda x: sum(x))
    

    available_stations["estimated_wait_time_to_charge"] = available_stations["trip_time_to_charger"] + available_stations["estimated_wait_time"]
    best_station = available_stations.nsmallest(1, "estimated_wait_time_to_charge").iloc[0]
    best_station_idx = best_station.index[0]
    
    try:
        empty_slot = best_station['occupied_slot'].index(0)
        fleet.loc[car_id, "status"] = "charging"
        fleet.loc[car_id, "charging_station_id"] = station_id

        
        charging_stations.loc[best_station_idx, 'occupied_slot'][empty_slot] = car_id
        charging_stations.loc[best_station_idx, 'estimated_time_to_charge'][empty_slot] = charging_time
        
    except:
        charging_stations.loc[best_station_idx, 'waiting_to_be_charged'].append(car_id)

7